# Beautiful Soup

[Site to scrape](https://en.wikipedia.org/wiki/Lists_of_legendary_creatures)

In [1]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd
import time

In [2]:
url = "https://en.wikipedia.org/wiki/Lists_of_legendary_creatures"

In [3]:
text = requests.get(url).text

In [4]:
soup = BeautifulSoup(text, 'html.parser')

In [5]:
type(soup)

bs4.BeautifulSoup

In [6]:
alphaList = soup.find_all('ul')[1]
print(alphaList.text)

List of legendary creatures (A)
List of legendary creatures (B)
List of legendary creatures (C)
List of legendary creatures (D)
List of legendary creatures (E)
List of legendary creatures (F)
List of legendary creatures (G)
List of legendary creatures (H)
List of legendary creatures (I)
List of legendary creatures (J)
List of legendary creatures (K)
List of legendary creatures (L)
List of legendary creatures (M)
List of legendary creatures (N)
List of legendary creatures (O)
List of legendary creatures (P)
List of legendary creatures (Q)
List of legendary creatures (R)
List of legendary creatures (S)
List of legendary creatures (T)
List of legendary creatures (U)
List of legendary creatures (V)
List of legendary creatures (W)
List of legendary creatures (X)
List of legendary creatures (Y)
List of legendary creatures (Z)


In [7]:
listPageLinks = [a['href'] for a in alphaList.find_all('a')]
listPageLinks

['/wiki/List_of_legendary_creatures_(A)',
 '/wiki/List_of_legendary_creatures_(B)',
 '/wiki/List_of_legendary_creatures_(C)',
 '/wiki/List_of_legendary_creatures_(D)',
 '/wiki/List_of_legendary_creatures_(E)',
 '/wiki/List_of_legendary_creatures_(F)',
 '/wiki/List_of_legendary_creatures_(G)',
 '/wiki/List_of_legendary_creatures_(H)',
 '/wiki/List_of_legendary_creatures_(I)',
 '/wiki/List_of_legendary_creatures_(J)',
 '/wiki/List_of_legendary_creatures_(K)',
 '/wiki/List_of_legendary_creatures_(L)',
 '/wiki/List_of_legendary_creatures_(M)',
 '/wiki/List_of_legendary_creatures_(N)',
 '/wiki/List_of_legendary_creatures_(O)',
 '/wiki/List_of_legendary_creatures_(P)',
 '/wiki/List_of_legendary_creatures_(Q)',
 '/wiki/List_of_legendary_creatures_(R)',
 '/wiki/List_of_legendary_creatures_(S)',
 '/wiki/List_of_legendary_creatures_(T)',
 '/wiki/List_of_legendary_creatures_(U)',
 '/wiki/List_of_legendary_creatures_(V)',
 '/wiki/List_of_legendary_creatures_(W)',
 '/wiki/List_of_legendary_creature

In [8]:
newUrl = f'https://en.wikipedia.org{listPageLinks[0]}'
print(newUrl)

https://en.wikipedia.org/wiki/List_of_legendary_creatures_(A)


In [9]:
newSoup = BeautifulSoup(requests.get(newUrl).text, 'html.parser')
newSoup

<!DOCTYPE html>

<html class="client-nojs" dir="ltr" lang="en">
<head>
<meta charset="utf-8"/>
<title>List of legendary creatures (A) - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":false,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"5dc19667-0000-4b5c-806b-835c806f9d1c","wgCSPNonce":false,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":false,"wgNamespaceNumber":0,"wgPageName":"List_of_legendary_creatures_(A)","wgTitle":"List of legendary creatures (A)","wgCurRevisionId":1076751995,"wgRevisionId":1076751995,"wgArticleId":24180353,"wgIsArticle":true,"wgIsRedirect":false,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Lists of legenda

In [10]:
newSoup.find_all('ul')[0]

<ul><li><a class="mw-selflink selflink">A</a></li>
<li><a href="/wiki/List_of_legendary_creatures_(B)" title="List of legendary creatures (B)">B</a></li>
<li><a href="/wiki/List_of_legendary_creatures_(C)" title="List of legendary creatures (C)">C</a></li>
<li><a href="/wiki/List_of_legendary_creatures_(D)" title="List of legendary creatures (D)">D</a></li>
<li><a href="/wiki/List_of_legendary_creatures_(E)" title="List of legendary creatures (E)">E</a></li>
<li><a href="/wiki/List_of_legendary_creatures_(F)" title="List of legendary creatures (F)">F</a></li>
<li><a href="/wiki/List_of_legendary_creatures_(G)" title="List of legendary creatures (G)">G</a></li>
<li><a href="/wiki/List_of_legendary_creatures_(H)" title="List of legendary creatures (H)">H</a></li>
<li><a href="/wiki/List_of_legendary_creatures_(I)" title="List of legendary creatures (I)">I</a></li>
<li><a href="/wiki/List_of_legendary_creatures_(J)" title="List of legendary creatures (J)">J</a></li>
<li><a href="/wiki/Lis

In [11]:
creatureList = newSoup.find_all('ul')[1]

In [12]:
testCreature = creatureList.find_all('li')[0]
testCreature

<li><a href="/wiki/%C3%81_Bao_A_Qu" title="Á Bao A Qu">Á Bao A Qu</a> (<a class="mw-redirect" href="/wiki/Malay_(ethnic_group)" title="Malay (ethnic group)">Malay</a>) – Entity that lives in the <a href="/wiki/Vijaya_Stambha" title="Vijaya Stambha">Tower of Victory in Chitor.</a></li>

In [13]:
testCreature.text

'Á Bao A Qu (Malay) – Entity that lives in the Tower of Victory in Chitor.'

In [14]:
testCreature.__str__()

'<li><a href="/wiki/%C3%81_Bao_A_Qu" title="Á Bao A Qu">Á Bao A Qu</a> (<a class="mw-redirect" href="/wiki/Malay_(ethnic_group)" title="Malay (ethnic group)">Malay</a>) – Entity that lives in the <a href="/wiki/Vijaya_Stambha" title="Vijaya Stambha">Tower of Victory in Chitor.</a></li>'

In [15]:
name = testCreature.find('a').text
name

'Á Bao A Qu'

In [16]:
pageLink = testCreature.find('a')['href']
pageLink

'/wiki/%C3%81_Bao_A_Qu'

In [18]:
origin = re.findall(r"\((.*)\)", testCreature.__str__())[0]
origin

'<a class="mw-redirect" href="/wiki/Malay_(ethnic_group)" title="Malay (ethnic group)">Malay</a>'

In [19]:
originSoup = BeautifulSoup(origin)
originName = originSoup.find('a').text
originName

'Malay'

In [20]:
originLink = originSoup.find('a')['href']
originLink

'/wiki/Malay_(ethnic_group)'

In [33]:
def parse_creature(creature):
    name = creature.find('a').text
    pageLink = creature.find('a')['href']
    origin = re.findall(r"\((.*)\)", creature.__str__())[0]
    originSoup = BeautifulSoup(origin)
    originNames = [a.text for a in originSoup.find_all('a')]
    originLinks = [a['href'] for a in originSoup.find_all('a')]
    return {
        "name": name,
        "pageLink": pageLink,
        "originNames": originNames,
        "originLinks": originLinks
    }

In [22]:
parse_creature(testCreature)

{'name': 'Á Bao A Qu',
 'pageLink': '/wiki/%C3%81_Bao_A_Qu',
 'originNames': ['Malay'],
 'originLinks': ['/wiki/Malay_(ethnic_group)']}

In [23]:
for creature in creatureList.find_all('li'):
    print(parse_creature(creature))

{'name': 'Á Bao A Qu', 'pageLink': '/wiki/%C3%81_Bao_A_Qu', 'originNames': ['Malay'], 'originLinks': ['/wiki/Malay_(ethnic_group)']}
{'name': 'Aatxe', 'pageLink': '/wiki/Aatxe', 'originNames': ['Basque'], 'originLinks': ['/wiki/Basque_mythology']}
{'name': 'Abaasy', 'pageLink': '/wiki/Abaasy', 'originNames': ['Yakuts'], 'originLinks': ['/wiki/Yakuts']}
{'name': 'Abada', 'pageLink': '/wiki/Abada_(unicorn)', 'originNames': ['African'], 'originLinks': ['/wiki/African_mythology']}
{'name': 'Äbädä', 'pageLink': '/wiki/%C3%84b%C3%A4d%C3%A4', 'originNames': ['Tatar'], 'originLinks': ['/wiki/Tatar_mythology']}
{'name': 'Abaia', 'pageLink': '/wiki/Abaia', 'originNames': ['Melanesia'], 'originLinks': ['/wiki/Melanesian_mythology']}
{'name': 'Abarimon', 'pageLink': '/wiki/Abarimon', 'originNames': ['Medieval Bestiaries'], 'originLinks': ['/wiki/Bestiary']}
{'name': 'Abath', 'pageLink': '/wiki/Abath', 'originNames': ['Malay'], 'originLinks': ['/wiki/Malay_(ethnic_group)']}
{'name': 'Abura-sumashi'

In [24]:
aCreatures = [parse_creature(creature) for creature in creatureList.find_all('li')]

In [25]:
pd.DataFrame(aCreatures)

,name,pageLink,originNames,originLinks
0,Á Bao A Qu,/wiki/%C3%81_Bao_A_Qu,[Malay],[/wiki/Malay_(ethnic_group)]
1,Aatxe,/wiki/Aatxe,[Basque],[/wiki/Basque_mythology]
2,Abaasy,/wiki/Abaasy,[Yakuts],[/wiki/Yakuts]
3,Abada,/wiki/Abada_(unicorn),[African],[/wiki/African_mythology]
4,Äbädä,/wiki/%C3%84b%C3%A4d%C3%A4,[Tatar],[/wiki/Tatar_mythology]
...,...,...,...,...
120,Ayakashi,/wiki/List_of_legendary_creatures_from_Japan#A,[Japanese],[/wiki/Japanese_mythology]
121,Ayakashi-no-ayashibi,/wiki/List_of_legendary_creatures_from_Japan#A,[Japanese],[/wiki/Japanese_mythology]
122,Aziza,/wiki/Aziza_(African_mythology),[Dahomey],[/wiki/Dahomey_mythology]
123,Azukiarai,/wiki/Azukiarai,[Japanese],[/wiki/Japanese_mythology]


In [26]:
def scrape_creature_page(url):
    R = requests.get(url)
    soup = BeautifulSoup(R.text, 'html.parser')
    creatureList = soup.find_all('ul')[1]
    creatures = [parse_creature(creature) for creature in creatureList.find_all('li')]
    return creatures

In [34]:
scrape_creature_page(f'https://en.wikipedia.org{listPageLinks[2]}')

IndexError: list index out of range

In [37]:
def parse_creature(creature):
    name = creature.find('a').text
    pageLink = creature.find('a')['href']
    try:
        origin = re.findall(r"\((.*)\)", creature.__str__())[0]
        originSoup = BeautifulSoup(origin)
        originNames = [a.text for a in originSoup.find_all('a')]
        originLinks = [a['href'] for a in originSoup.find_all('a')]
    except IndexError:
        originNames = []
        originLinks = []
        print(f"No origin found in {creature.__str__()}")
    return {
        "name": name,
        "pageLink": pageLink,
        "originNames": originNames,
        "originLinks": originLinks
    }

In [38]:
scrape_creature_page(f'https://en.wikipedia.org{listPageLinks[2]}')

No origin found in <li><a href="/wiki/Bicorn_and_Chichevache" title="Bicorn and Chichevache">Chichevache</a> – Human-faced cow that feeds on good women</li>


[{'name': 'Cabeiri',
  'pageLink': '/wiki/Cabeiri',
  'originNames': ['Greek'],
  'originLinks': ['/wiki/Greek_mythology']},
 {'name': 'Cacus',
  'pageLink': '/wiki/Cacus',
  'originNames': ['Roman'],
  'originLinks': ['/wiki/Roman_mythology']},
 {'name': 'Cadejo',
  'pageLink': '/wiki/Cadejo',
  'originNames': ['Central America'],
  'originLinks': ['/wiki/Central_America']},
 {'name': 'Cailleach',
  'pageLink': '/wiki/Cailleach',
  'originNames': ['Scottish'],
  'originLinks': ['/wiki/Scottish_folklore']},
 {'name': 'Caipora',
  'pageLink': '/wiki/Caipora',
  'originNames': ['Tupi'],
  'originLinks': ['/wiki/Tupi_people']},
 {'name': 'Caladrius',
  'pageLink': '/wiki/Caladrius',
  'originNames': ['Medieval Bestiary'],
  'originLinks': ['/wiki/Bestiary']},
 {'name': 'Calingi',
  'pageLink': '/wiki/Calingi',
  'originNames': ['Medieval Bestiary'],
  'originLinks': ['/wiki/Bestiary']},
 {'name': 'Callitrix',
  'pageLink': '/wiki/Callitrix',
  'originNames': ['Medieval Bestiary'],
  'orig

In [32]:
allCreatures = []
for link in listPageLinks:
    allCreatures.extend(scrape_creature_page(f"https://en.wikipedia.org{link}"))
    time.sleep(0.2)

No origin found in <li><a href="/wiki/Blafard" title="Blafard">Blafard</a> – Imaginary creature from the early <a class="mw-redirect" href="/wiki/United_States_of_America" title="United States of America">United States of America</a></li>
No origin found in <li><a href="/wiki/Bicorn_and_Chichevache" title="Bicorn and Chichevache">Chichevache</a> – Human-faced cow that feeds on good women</li>


/Users/jguillette/Library/Caches/pypoetry/virtualenvs/python-web-scraping-workshop-nQZGi0iI-py3.9/lib/python3.9/site-packages/bs4/__init__.py:435: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  warnings.warn(


No origin found in <li><a href="/wiki/Ghost" title="Ghost">Ghost</a> – Disembodied spirits of those that have died</li>
No origin found in <li><a href="/wiki/Hoopoe#Hoopoes_in_human_culture" title="Hoopoe">Hoopoe</a> – <a href="/wiki/Near_passerine" title="Near passerine">near passerine</a> bird common to <a href="/wiki/Africa" title="Africa">Africa</a> and <a href="/wiki/Eurasia" title="Eurasia">Eurasia</a> that features in many mythologies in those continents</li>
No origin found in <li><a href="/wiki/Hoop_snake" title="Hoop snake">Hoop snake</a> – snake which rolls by taking its tail in its mouth</li>
No origin found in <li><a href="/wiki/Lava_bear" title="Lava bear">Lava bear</a> – Miniature bear thought to inhabit the lava beds of south central Oregon</li>
No origin found in <li><a href="/wiki/Chinese_dragon" title="Chinese dragon">Lóng</a> – <a href="/wiki/Chinese_mythology" title="Chinese mythology">Chinese</a> dragon</li>
No origin found in <li><a href="/wiki/Lusca" title="Lusc

AttributeError: 'NoneType' object has no attribute 'text'

In [39]:
def parse_creature(creature):
    try:
        name = creature.find('a').text
        pageLink = creature.find('a')['href']
    except AttributeError:
        name = ""
        pageLink = ""
        print(f"Could not find link in {creature}")
    try:
        origin = re.findall(r"\((.*)\)", creature.__str__())[0]
        originSoup = BeautifulSoup(origin)
        originNames = [a.text for a in originSoup.find_all('a')]
        originLinks = [a['href'] for a in originSoup.find_all('a')]
    except IndexError:
        originNames = []
        originLinks = []
        print(f"No origin found in {creature}")
    return {
        "name": name,
        "pageLink": pageLink,
        "originNames": originNames,
        "originLinks": originLinks
    }

In [40]:
allCreatures = []
for link in listPageLinks:
    allCreatures.extend(scrape_creature_page(f"https://en.wikipedia.org{link}"))
    time.sleep(0.2)

No origin found in <li><a href="/wiki/Blafard" title="Blafard">Blafard</a> – Imaginary creature from the early <a class="mw-redirect" href="/wiki/United_States_of_America" title="United States of America">United States of America</a></li>
No origin found in <li><a href="/wiki/Bicorn_and_Chichevache" title="Bicorn and Chichevache">Chichevache</a> – Human-faced cow that feeds on good women</li>
No origin found in <li><a href="/wiki/Ghost" title="Ghost">Ghost</a> – Disembodied spirits of those that have died</li>
No origin found in <li><a href="/wiki/Hoopoe#Hoopoes_in_human_culture" title="Hoopoe">Hoopoe</a> – <a href="/wiki/Near_passerine" title="Near passerine">near passerine</a> bird common to <a href="/wiki/Africa" title="Africa">Africa</a> and <a href="/wiki/Eurasia" title="Eurasia">Eurasia</a> that features in many mythologies in those continents</li>
No origin found in <li><a href="/wiki/Hoop_snake" title="Hoop snake">Hoop snake</a> – snake which rolls by taking its tail in its mou

In [41]:
def parse_creature(creature):
    try:
        name = creature.find('a').text
        pageLink = creature.find('a')['href']
    except AttributeError:
        name = ""
        pageLink = ""
        print(f"Could not find link in {creature}")
    try:
        origin = re.findall(r"\((.*)\)", creature.__str__())[0]
        originSoup = BeautifulSoup(origin)
        originNames = [a.text for a in originSoup.find_all('a')]
        originLinks = [a['href'] for a in originSoup.find_all('a')]
    except IndexError:
        originNames = []
        originLinks = []
        print(f"No origin found in {creature}")
    return {
        "name": name,
        "pageLink": pageLink,
        "originNames": originNames,
        "originLinks": originLinks,
        "sourceText": creature.__str__()
    }

In [42]:
allCreatures = []
for link in listPageLinks:
    allCreatures.extend(scrape_creature_page(f"https://en.wikipedia.org{link}"))
    time.sleep(0.2)

No origin found in <li><a href="/wiki/Blafard" title="Blafard">Blafard</a> – Imaginary creature from the early <a class="mw-redirect" href="/wiki/United_States_of_America" title="United States of America">United States of America</a></li>
No origin found in <li><a href="/wiki/Bicorn_and_Chichevache" title="Bicorn and Chichevache">Chichevache</a> – Human-faced cow that feeds on good women</li>
No origin found in <li><a href="/wiki/Ghost" title="Ghost">Ghost</a> – Disembodied spirits of those that have died</li>
No origin found in <li><a href="/wiki/Hoopoe#Hoopoes_in_human_culture" title="Hoopoe">Hoopoe</a> – <a href="/wiki/Near_passerine" title="Near passerine">near passerine</a> bird common to <a href="/wiki/Africa" title="Africa">Africa</a> and <a href="/wiki/Eurasia" title="Eurasia">Eurasia</a> that features in many mythologies in those continents</li>
No origin found in <li><a href="/wiki/Hoop_snake" title="Hoop snake">Hoop snake</a> – snake which rolls by taking its tail in its mou

In [43]:
pd.DataFrame(allCreatures)

,name,pageLink,originNames,originLinks,sourceText
0,Á Bao A Qu,/wiki/%C3%81_Bao_A_Qu,[Malay],[/wiki/Malay_(ethnic_group)],"<li><a href=""/wiki/%C3%81_Bao_A_Qu"" title=""Á B..."
1,Aatxe,/wiki/Aatxe,[Basque],[/wiki/Basque_mythology],"<li><a href=""/wiki/Aatxe"" title=""Aatxe"">Aatxe<..."
2,Abaasy,/wiki/Abaasy,[Yakuts],[/wiki/Yakuts],"<li><a class=""mw-redirect"" href=""/wiki/Abaasy""..."
3,Abada,/wiki/Abada_(unicorn),[African],[/wiki/African_mythology],"<li><a href=""/wiki/Abada_(unicorn)"" title=""Aba..."
4,Äbädä,/wiki/%C3%84b%C3%A4d%C3%A4,[Tatar],[/wiki/Tatar_mythology],"<li><a href=""/wiki/%C3%84b%C3%A4d%C3%A4"" title..."
...,...,...,...,...,...
1446,Zmiy,/wiki/Slavic_dragon,[],[],"<li><a href=""/wiki/Slavic_dragon"" title=""Slavi..."
1447,Zombie,/wiki/Zombie,[Vodou],[/wiki/Haitian_Vodou],"<li><a href=""/wiki/Zombie"" title=""Zombie"">Zomb..."
1448,Zorigami,/wiki/Tsukumogami#Types_of_tsukumogami,[Japanese],[/wiki/Japanese_mythology],"<li><a href=""/wiki/Tsukumogami#Types_of_tsukum..."
1449,Zuijin,/wiki/Zuijin,[Japanese],[/wiki/Japanese_mythology],"<li><a href=""/wiki/Zuijin"" title=""Zuijin"">Zuij..."
